*   **Thành viên 1: 519H0148 - Phạm Đỗ Trọng Đại**
*   **Thành viên 2: 521H0326 - Nguyễn Anh Tuấn**



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
AMZN_df = pd.read_csv('AMZN.csv', parse_dates = ['Date'], index_col = 0)

In [ ]:
# In ra kích thước của DataFrame 'AMZN_df'
print(AMZN_df.shape)

In [ ]:
# Hiển thị 5 dòng đầu DataFrame
AMZN_df.head()

In [ ]:
# Hiển thị 5 dòng cuối DataFrame
AMZN_df.tail()

In [ ]:
#in ra tổng số lượng giá trị null trong DataFrame
print(AMZN_df.isnull().values.sum())
print()

In [ ]:
#kiểm tra xem có giá trị bằng 0 trong DataFrame không
print(AMZN_df.eq(0).any().any())

In [ ]:
print(sum(AMZN_df['Close'] - AMZN_df['Adj Close']))

In [ ]:
#loại bỏ cột 'Adj Close' từ DataFrame
AMZN_df.drop('Adj Close' , inplace = True , axis = 1)

In [ ]:
print(AMZN_df.head())
print()

# EDA(Exploratory Data Analysis)

### **Thực hiện phân tích về (Open, High, Low, Close)**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Tạo một subplot với kích thước 20x10
fig, ax = plt.subplots(figsize=(20,10))
# Vẽ đồ thị đường sử dụng seaborn, dữ liệu là cột 'Open' của DataFrame AMZN_df
sns.lineplot(data=AMZN_df['Open'],linewidth=1)
plt.legend(['Amazon'])
plt.title("'Open' stock prices from 2015 to 2020")
plt.xlabel('Year')

In [ ]:
# Tạo một subplot với kích thước 20x10
fig, ax = plt.subplots(figsize=(20,10))
# Vẽ đồ thị đường sử dụng seaborn, dữ liệu là cột 'High' của DataFrame AMZN_df
sns.lineplot(data=AMZN_df['High'],linewidth=1)
plt.legend(['Amazon'])
plt.title("'High' stock prices from 2015 to 2020")
plt.xlabel('Year')

In [ ]:
# Tạo một subplot với kích thước 20x10
fig, ax = plt.subplots(figsize=(20,10))
# Vẽ đồ thị đường sử dụng seaborn, dữ liệu là cột 'Low' của DataFrame AMZN_df
sns.lineplot(data=AMZN_df['Low'],linewidth=1)
plt.legend(['Amazon'])
plt.title("'Low' stock prices from 2015 to 2020")
plt.xlabel('Year')

In [ ]:
# Tạo một subplot với kích thước 20x10
fig, ax = plt.subplots(figsize=(20,10))
# Vẽ đồ thị đường sử dụng seaborn, dữ liệu là cột 'Close' của DataFrame AMZN_df
sns.lineplot(data=AMZN_df['Close'],linewidth=1)
plt.legend(['Amazon'])
plt.title("'Close' stock prices from 2015 to 2020")
plt.xlabel('Year')

### **Thực hiện phân tích về (Volumn)**

In [ ]:
# Tạo một figure và một trục sử dụng subplots, với kích thước (20, 10)
fig, ax = plt.subplots(figsize=(20,10))
# Sử dụng seaborn để vẽ đồ thị đường ('lineplot') với dữ liệu từ cột 'Volume' của DataFrame
sns.lineplot(data=AMZN_df['Volume'],linewidth=1)
plt.legend(['Amazon'])
plt.title("'Volume' from 2015 to 2020")
plt.xlabel('Year')

### **Hệ số tương quan**

In [ ]:
#Tạo 1 figure với 1 cột
fig, axs = plt.subplots(ncols=1,figsize=(20,12))
# Vẽ heatmap của ma trận tương quan của DataFrame
sns.heatmap(AMZN_df.corr(), annot=True, cmap = plt.cm.CMRmap_r, linecolor='white', linewidth=2, ax=axs)
axs.title.set_text('Amazon feature correlation')
plt.show()

**Feature Open, high, low có mối tương quan cao với feature Close. Chúng ta có thể sử dụng một trong hai đặc điểm này để dự đoán nhằm tránh hiện tượng đa cộng tuyến**

# Chuẩn hóa dữ liệu

In [ ]:
# Chuẩn hóa dữ liệu sử dụng MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

## AMAZON

In [ ]:
AMZN_close_stock = AMZN_df['Close']
#thực hiện chuẩn hóa giá đóng cửa của cổ phiếu
AMZN_close_stock_scaled = sc.fit_transform(np.array(AMZN_close_stock).reshape(-1,1))
AMZN_close_stock_scaled

In [ ]:
#Tính toán kích thước của tập huấn luyện bằng cách lấy 65% của tổng số lượng mẫu dữ liệu trong chuỗi thời gian của giá đóng cửa của cổ phiếu Amazon
AMZN_train_size = int(len(AMZN_close_stock_scaled)*0.65)
#Tính toán kích thước của tập kiểm thử bằng cách lấy phần còn lại 35% của tổng số lượng mẫu dữ liệu sau khi đã chọn tập huấn luyện.
AMZN_test_size = len(AMZN_close_stock_scaled) - AMZN_train_size
#Chia dữ liệu thành tập huấn luyện và tập kiểm thử
AMZN_train_set, AMZN_test_set = AMZN_close_stock_scaled[0:AMZN_train_size, :], AMZN_close_stock_scaled[AMZN_train_size:len(AMZN_close_stock_scaled), :1]

# Tạo train data, test data

In [ ]:
def load_data(dataset, timesteps = 1):
  X = []
  y = []
  for i in range(timesteps, len(dataset)):
    X.append(dataset[i-timesteps:i, 0])
    y.append(dataset[i, 0])

  return np.array(X), np.array(y)


In [ ]:
timesteps = 100

AMAZON_x_train, AMAZON_y_train = load_data(AMZN_train_set,timesteps)
AMAZON_x_test, AMAZON_y_test = load_data(AMZN_test_set,timesteps)
# print(AMAZON_x_train)
print(AMAZON_x_train.shape)
# print(AMAZON_y_train)
print(AMAZON_y_train.shape)

In [ ]:
# Định hình lại
AMAZON_x_train = np.reshape(AMAZON_x_train, (AMAZON_x_train.shape[0], AMAZON_x_train.shape[1], 1))
AMAZON_x_test = np.reshape(AMAZON_x_test, (AMAZON_x_test.shape[0], AMAZON_x_test.shape[1], 1))

# Model and validate data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, SimpleRNN

In [ ]:
# sử dụng thư viện Keras để xây dựng một mô hình mạng nơ-ron sử dụng lớp SimpleRNN
def SimpleRNN_model_without_dropout(trainX, trainy):
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], 1
  input_shape=(n_timesteps,n_features)
  print (input_shape)
  model = Sequential()
  # Thêm lớp RNN đầu tiên
  model.add(SimpleRNN(units = 50, activation='tanh', return_sequences = True, input_shape = input_shape))

  # Thêm lớp RNN đầu thứ 2
  model.add(SimpleRNN(units = 50, return_sequences = True))

  # Thêm lớp LSTM thứ ba
  model.add(SimpleRNN(units = 50))

  # Thêm lớp đầu ra
  model.add((Dense(n_outputs)))
  return model

In [ ]:
#sử dụng thư viện Keras để định nghĩa một mô hình SimpleRNN với lớp Dropout.
def SimpleRNN_model_dropout(trainX, trainy):
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], 1
  input_shape=(n_timesteps,n_features)
  print (input_shape)
  model = Sequential()
  # Thêm lớp RNN đầu tiên và một số chính quy Dropout
  model.add(SimpleRNN(units = 50, activation='tanh', return_sequences = True, input_shape = input_shape))
  model.add(Dropout(0.2))

  # Thêm lớp RNN thứ 2 và một số chính quy Dropout
  model.add(SimpleRNN(units = 50, return_sequences = True))
  model.add(Dropout(0.2))

  # Thêm lớp LSTM thứ ba và một số chính quy Dropout
  model.add(SimpleRNN(units = 50))
  model.add(Dropout(0.2))
  # Thêm lớp đầu ra
  model.add((Dense(n_outputs)))
  return model

### **AMAZON**

## RNN

#### **RNN không dùng Regularization method**

In [ ]:
# Tạo và huấn luyện mô hình RNN
AMAZON_model=SimpleRNN_model_without_dropout(AMAZON_x_train, AMAZON_y_train)

In [ ]:
AMAZON_model.summary()

In [ ]:
# Biên dịch RNN
AMAZON_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# đo lường thời gian thực hiện quá trình huấn luyện.
import datetime
begin_time = datetime.datetime.now()
epochs, batch_size = 100, 32
history=AMAZON_model.fit(AMAZON_x_train, AMAZON_y_train, shuffle=True, validation_split = 0.1, epochs=epochs, batch_size=batch_size, verbose=1)
print(datetime.datetime.now() - begin_time)

##### **Thực hiện dự đoán và kiểm tra số liệu hiệu suất**

In [ ]:
# plot loss trong quá trình huấn luyện
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# kiểm tra thời gian chạy của hai pha dự đoán trên tập huấn luyện và tập kiểm thử và in ra kết quả
begin_time = datetime.datetime.now()
train_pred = AMAZON_model.predict(AMAZON_x_train)
test_pred = AMAZON_model.predict(AMAZON_x_test)
print(datetime.datetime.now() - begin_time)

In [ ]:
# chuyển đổi các dự đoán được chuẩn hóa (scaled predictions) trở lại đơn vị thực tế
train_pred_inv = sc.inverse_transform(train_pred.copy())
test_pred_inv = sc.inverse_transform(test_pred.copy())
AMAZON_y_train_inv = sc.inverse_transform(np.reshape(AMAZON_y_train,(len(AMAZON_y_train),1)))
AMAZON_y_test_inv = sc.inverse_transform(np.reshape(AMAZON_y_test,(len(AMAZON_y_test),1)))

In [ ]:
# chỉ số đánh giá
mse = np.mean(((train_pred_inv - AMAZON_y_train_inv) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

In [ ]:
mse = np.mean(((test_pred_inv - AMAZON_y_test_inv) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

In [ ]:
# đánh giá hiệu suất của mô hình học máy (machine learning) trên tập kiểm thử (test set)
from sklearn.metrics import accuracy_score
# kiểm tra mô hình dự đoán trên 10 mẫu đầu tiên trong tập kiểm thử
for i in range(10):
    print(test_pred_inv[i], AMAZON_y_test_inv[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(AMAZON_y_test_inv, color = 'black', label = 'Test')
plt.plot(test_pred_inv, color = 'green', label = 'pred')
plt.title('SimpleRNN without DROPOUT REGULARIZATION')
plt.legend()
plt.show()

#### **RNN sử dụng Regularization method**

In [ ]:
# Tạo và huấn luyện mô hình RNN với kĩ thuật Dropout
AMAZON_model_dropout=SimpleRNN_model_dropout(AMAZON_x_train, AMAZON_y_train)

In [ ]:
AMAZON_model_dropout.summary()

In [ ]:
# Biên dịch RNN
AMAZON_model_dropout.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# đo lường thời gian thực hiện quá trình huấn luyện.
import datetime
begin_time = datetime.datetime.now()
epochs, batch_size = 100, 32
history=AMAZON_model_dropout.fit(AMAZON_x_train, AMAZON_y_train, shuffle=True, validation_split = 0.1, epochs=epochs, batch_size=batch_size, verbose=1)
print(datetime.datetime.now() - begin_time)

##### **Do the prediction and check performance metrics**

In [ ]:
# plot loss trong quá trình huấn luyện
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# kiểm tra thời gian chạy của hai pha dự đoán trên tập huấn luyện và tập kiểm thử và in ra kết quả
begin_time = datetime.datetime.now()
train_pred = AMAZON_model_dropout.predict(AMAZON_x_train)
test_pred = AMAZON_model_dropout.predict(AMAZON_x_test)
print(datetime.datetime.now() - begin_time)

In [ ]:
# chuyển đổi các dự đoán được chuẩn hóa (scaled predictions) trở lại đơn vị thực tế
train_pred_inv = sc.inverse_transform(train_pred.copy())
test_pred_inv = sc.inverse_transform(test_pred.copy())
AMAZON_y_train_inv = sc.inverse_transform(np.reshape(AMAZON_y_train,(len(AMAZON_y_train),1)))
AMAZON_y_test_inv = sc.inverse_transform(np.reshape(AMAZON_y_test,(len(AMAZON_y_test),1)))

In [ ]:
# SimpleRNN với Dropout regularization
# chỉ số đánh giá
mse = np.mean(((train_pred_inv - AMAZON_y_train_inv) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

In [ ]:
mse = np.mean(((test_pred_inv - AMAZON_y_test_inv) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

In [ ]:
# đánh giá hiệu suất của mô hình học máy (machine learning) trên tập kiểm thử (test set)
from sklearn.metrics import accuracy_score
# kiểm tra mô hình dự đoán trên 10 mẫu đầu tiên trong tập kiểm thử
for i in range(10):
    print(test_pred_inv[i], AMAZON_y_test_inv[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(AMAZON_y_test_inv, color = 'black', label = 'Test')
plt.plot(test_pred_inv, color = 'green', label = 'pred')
plt.title('SimpleRNN with DROPOUT REGULARIZATION')
plt.legend()
plt.show()

# LSTM

In [ ]:
# tạo mô hình và đào tạo
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, Flatten
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError
model = tf.keras.Sequential()

In [ ]:
# Lớp LSTM
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.5))

#Lớp cuối cùng
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae','mse','accuracy'])

print("Amazone: ")
history = model.fit(AMAZON_x_train, AMAZON_y_train, validation_data=(AMAZON_x_test,AMAZON_y_test), epochs=40,batch_size=40, verbose=1, shuffle =True)


In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
plt.plot(history.history['mae'], label='train mae')
plt.plot(history.history['val_mae'], label='val mae')
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
# Sau khi xây dựng xong mô hình, chúng ta sẽ tóm tắt lại
from tensorflow.keras.utils import plot_model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# đánh giá hiệu suất của mô hình trên tập dữ liệu kiểm tra.
model.evaluate(AMAZON_x_test, AMAZON_y_test)


In [ ]:
from sklearn.metrics import explained_variance_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import r2_score
from sklearn.metrics import max_error

# dự đoán xác suất cho tập kiểm tra
yhat_probs = model.predict(AMAZON_x_test, verbose=0)
# giảm xuống mảng 1d
yhat_probs = yhat_probs[:, 0]

var = explained_variance_score(AMAZON_y_test.reshape(-1,1), yhat_probs)
print('Variance: %f' % var)

r2 = r2_score(AMAZON_y_test.reshape(-1,1), yhat_probs)
print('R2 Score: %f' % var)

var2 = max_error(AMAZON_y_test.reshape(-1,1), yhat_probs)
print('Max Error: %f' % var2)

In [ ]:
predicted  = model.predict(AMAZON_x_test)
test_label = AMAZON_y_test.reshape(-1,1)
predicted = np.array(predicted[:,0]).reshape(-1,1)
len_t = len(AMAZON_x_train)
#Lặp qua từng phần tử trong dữ liệu test.
for j in range(len_t , len_t + len(AMAZON_x_test)):
    temp = AMZN_df.iloc[j,3]
    test_label[j - len_t] = test_label[j - len_t] * temp + temp
    predicted[j - len_t] = predicted[j - len_t] * temp + temp
# in ra đồ thị
plt.plot(predicted, color = 'green', label = 'Predicted  Stock Price')
plt.plot(test_label, color = 'red', label = 'Real Stock Price')
plt.title(' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel(' Stock Price')
plt.legend()
plt.show()

##FFNN

In [ ]:
from sklearn.model_selection import train_test_split
df = AMZN_df
# lấy dữ liệu từ cột Close trong dataFrame
data = df['Close'].values.reshape(-1, 1)
# chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)
X, Y = [], []
window_size = 10  # Độ dài cửa sổ dữ liệu

for i in range(len(data_scaled) - window_size):
    X.append(data_scaled[i:i+window_size, 0])
    Y.append(data_scaled[i+window_size, 0])

X, Y = np.array(X), np.array(Y)

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
# in ra kích thước của tập huấn luyện và tập kiểm thử
print(len(X_train))
print(len(X_test))

In [ ]:
# xây dựng một mô hình mạng nơ-ron sử dụng thư viện TensorFlow và Keras
import tensorflow as tf
model = tf.keras.models.Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 1)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu', input_shape=(window_size,)))
model.add(tf.keras.layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
# Sau khi xây dựng xong mô hình, chúng ta sẽ tóm tắt lại
print(model.summary())
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
# đánh giá hiệu suất mô hình dựa trên giá trị Loss
loss = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
# in ra đồ thị
plt.figure(figsize=(12, 6))
plt.plot(df.index[-len(Y_test):], scaler.inverse_transform(Y_test.reshape(-1, 1)), label='Actual Price')
plt.plot(df.index[-len(Y_test):], predictions, label='Predicted Price', linestyle='dashed')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()